# **Прогнозирование статуса курильщика: нейронные сети**

В данном ноутбуке мы рассмотрим задачу бинарной классификации — предсказать, является ли человек курильщиком (`1`) или нет (`0`).  
Нам даны тренировочный и тестовый наборы данных, и итоговая метрика для оценки — *ROC AUC*.


---

<a id="step1"></a>
## **1. Загрузка библиотек и данных**


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Для разделения данных и оценки результатов
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# Различные скейлеры
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import copy

# Фиксируем random seed для воспроизводимости
def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")


Используемое устройство: cuda


In [4]:
# Загрузка данных
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')
sample_submission = pd.read_csv('/content/sample_submission.csv')

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

train_data.head()


Train shape: (15000, 24)
Test shape: (10000, 23)


,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,35.0,175.0,75.0,86.5,1.2,1.2,1.0,1.0,127.0,...,58.0,108.0,15.6,1.0,0.9,17.0,14.0,21.0,0.0,0.0
1,1,45.0,155.0,60.0,82.0,1.2,1.0,1.0,1.0,129.0,...,50.0,110.0,14.0,1.0,0.7,22.0,18.0,14.0,0.0,0.0
2,2,35.0,175.0,60.0,74.0,1.2,1.2,1.0,1.0,100.0,...,58.0,116.0,14.8,1.0,0.9,20.0,15.0,16.0,0.0,1.0
3,3,60.0,160.0,55.0,74.0,1.2,1.5,1.0,1.0,139.0,...,73.0,95.0,15.1,1.0,0.7,47.0,31.0,15.0,0.0,0.0
4,4,40.0,160.0,55.0,71.0,0.9,1.2,1.0,1.0,100.0,...,66.0,103.0,13.1,1.0,0.6,24.0,21.0,13.0,0.0,0.0


# **2. Предварительный анализ данных**

In [5]:
# Краткая информация о данных
train_data.info()

# Проверим баланс классов в train
print("\nРаспределение целевой переменной 'smoking':")
print(train_data['smoking'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15000 non-null  int64  
 1   age                  15000 non-null  float64
 2   height(cm)           15000 non-null  float64
 3   weight(kg)           15000 non-null  float64
 4   waist(cm)            15000 non-null  float64
 5   eyesight(left)       15000 non-null  float64
 6   eyesight(right)      15000 non-null  float64
 7   hearing(left)        15000 non-null  float64
 8   hearing(right)       15000 non-null  float64
 9   systolic             15000 non-null  float64
 10  relaxation           15000 non-null  float64
 11  fasting blood sugar  15000 non-null  float64
 12  Cholesterol          15000 non-null  float64
 13  triglyceride         15000 non-null  float64
 14  HDL                  15000 non-null  float64
 15  LDL                  15000 non-null 

In [6]:
train_data.describe()


,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,...,15000.000000,15000.000000,15000.000000,15000.000000,15000.00000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7499.500000,42.776000,164.764333,64.417333,80.407413,1.033813,1.028093,1.005867,1.005000,119.264067,...,57.320067,113.320133,14.516587,1.017800,0.86640,22.209533,20.899933,26.338667,0.145867,0.370467
std,4330.271354,10.436367,8.503976,11.000541,7.945482,0.316024,0.295498,0.076372,0.070536,11.202995,...,11.517555,18.674356,1.404844,0.169367,0.16628,5.578372,10.091596,19.714344,0.352984,0.482946
min,0.000000,20.000000,140.000000,35.000000,57.400000,0.000000,0.100000,1.000000,1.000000,86.000000,...,28.000000,36.000000,0.900000,1.000000,0.40000,9.000000,2.000000,6.000000,0.000000,0.000000
25%,3749.750000,40.000000,160.000000,55.000000,75.000000,0.800000,0.800000,1.000000,1.000000,110.000000,...,49.000000,100.000000,13.500000,1.000000,0.80000,18.000000,14.000000,15.000000,0.000000,0.000000
50%,7499.500000,40.000000,165.000000,65.000000,80.050000,1.000000,1.000000,1.000000,1.000000,119.000000,...,56.000000,113.000000,14.700000,1.000000,0.90000,21.000000,18.000000,21.000000,0.000000,0.000000
75%,11249.250000,50.000000,170.000000,70.000000,86.000000,1.200000,1.200000,1.000000,1.000000,128.000000,...,65.000000,126.000000,15.600000,1.000000,1.00000,25.000000,24.000000,31.000000,0.000000,1.000000
max,14999.000000,85.000000,190.000000,115.000000,114.000000,9.900000,9.900000,2.000000,2.000000,174.000000,...,106.000000,202.000000,20.000000,4.000000,1.50000,83.000000,139.000000,791.000000,1.000000,1.000000


## **3. Feat and Scaler**

In [35]:
# Список числовых признаков
num_cols = [
    'age', 'height(cm)', 'weight(kg)', 'waist(cm)',
    'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)',
    'systolic', 'relaxation', 'fasting blood sugar',
    'Cholesterol', 'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'Urine protein',
    'serum creatinine', 'AST', 'ALT', 'Gtp', 'dental caries'
    # 'smoking' тут не трогаем, это таргет
]

# Пример использования StandardScaler
scaler_standard = StandardScaler()

train_data_scaled_standard = train_data.copy()
train_data_scaled_standard[num_cols] = scaler_standard.fit_transform(train_data_scaled_standard[num_cols])

# То же самое можем сделать для других скейлеров, например RobustScaler:
scaler_robust = RobustScaler()
train_data_scaled_robust = train_data.copy()
train_data_scaled_robust[num_cols] = scaler_robust.fit_transform(train_data_scaled_robust[num_cols])


# **4. Dataset**

In [37]:
# Настраиваем DATASET
class SmokingDataset(Dataset):
    def __init__(self, df, feature_cols, target_col='smoking'):
        self.df = df.reset_index(drop=True)
        self.features = self.df[feature_cols].values
        self.targets = self.df[target_col].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = self.features[idx, :].astype(np.float32)
        y = int(self.targets[idx])
        return torch.tensor(X), torch.tensor(y)

# **5. DATALOADER**

In [8]:
def create_dataloaders(df, feature_cols, target_col, batch_size, scaler=None,
                       sampler_use=False, shuffle=False):
    """Удобная функция для создания DataLoader."""
    if scaler is not None:
        # Применяем указанный скейлер
        df_temp = df.copy()
        df_temp[feature_cols] = scaler.fit_transform(df_temp[feature_cols])
    else:
        df_temp = df

    dataset = SmokingDataset(df_temp, feature_cols, target_col)

    if sampler_use:
        # считаем веса классов
        labels = df_temp[target_col].values
        class_weights = 1. / torch.tensor(
            [(labels == t).sum() for t in np.unique(labels)], dtype=torch.float
        )
        sample_weights = class_weights[labels]
        sampler = torch.utils.data.WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )
        dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return dataloader




<a id="step6"></a>
## **6. Построение и обучение нейросетевых моделей**

<a id="step61"></a>
### **6.1 Baseline-модель (простой MLP)**

Для начала определим самую базовую архитектуру:
- Пара полносвязных слоёв,
- `ReLU` или `LeakyReLU`,
- Выход в 2 класса.

---


In [23]:
class BaselineNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, dropout=0.2):
        super(BaselineNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim//2)
        self.act2 = nn.ReLU()
        self.drop2 = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_dim//2, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.drop2(x)
        x = self.out(x)
        return x

def train_model(model, train_loader, val_loader, epochs=20, lr=1e-3,
                use_focal=False, alpha_focal=0.25, gamma_focal=2.0):
    """Функция обучения модели, возвращает обученную модель и историю."""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Критерий
    if use_focal:
        criterion = FocalLoss(alpha=alpha_focal, gamma=gamma_focal)
    else:
        criterion = nn.CrossEntropyLoss()

    model = model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_auc = 0.0

    train_history = []
    val_history = []

    for epoch in range(epochs):
        model.train()
        train_losses = []
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        # Оценка на train
        train_loss = np.mean(train_losses)

        # Оценка на val
        model.eval()
        val_losses = []
        all_probs = []
        all_labels = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_losses.append(loss.item())

                # Считаем вероятности класса 1
                probs = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                labels = y_batch.cpu().numpy()

                all_probs.extend(probs)
                all_labels.extend(labels)

        val_loss = np.mean(val_losses)
        val_auc = roc_auc_score(all_labels, all_probs)

        train_history.append(train_loss)
        val_history.append(val_loss)

        if val_auc > best_auc:
            best_auc = val_auc
            best_model_wts = copy.deepcopy(model.state_dict())

        if (epoch+1) % 5 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1}/{epochs}]",
                  f"Train Loss: {train_loss:.4f}",
                  f"Val Loss: {val_loss:.4f}",
                  f"Val ROC AUC: {val_auc:.4f}")

    print(f"\nBest ROC AUC on val: {best_auc:.4f}")
    model.load_state_dict(best_model_wts)
    return model, (train_history, val_history, best_auc)

# Реализуем FocalLoss (если хотим)
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        else:
            return focal_loss


Теперь запустим эксперимент с **BaselineNN** и, например, **StandardScaler**.

---


In [12]:
# Разделим данные на train/val
df_train, df_val = train_test_split(train_data, test_size=0.2, random_state=42, stratify=train_data['smoking'])

feature_cols = num_cols  # все числовые без 'smoking'
target_col = 'smoking'

# Создаём DataLoader
batch_size = 32
scaler = StandardScaler()  # попробуем сначала StandardScaler
train_loader = create_dataloaders(df_train, feature_cols, target_col,
                                  batch_size=batch_size, scaler=scaler,
                                  sampler_use=False, shuffle=True)
val_loader = create_dataloaders(df_val, feature_cols, target_col,
                                batch_size=batch_size, scaler=scaler,
                                sampler_use=False, shuffle=False)

# Инициализируем модель
model_baseline = BaselineNN(input_dim=len(feature_cols), hidden_dim=64, dropout=0.2)

# Обучаем
model_baseline, history_baseline = train_model(
    model_baseline,
    train_loader,
    val_loader,
    epochs=30,
    lr=1e-3,
    use_focal=False  # пока без FocalLoss
)


Epoch [1/30] Train Loss: 0.4554 Val Loss: 0.4057 Val ROC AUC: 0.8805
Epoch [5/30] Train Loss: 0.4068 Val Loss: 0.3967 Val ROC AUC: 0.8855
Epoch [10/30] Train Loss: 0.3969 Val Loss: 0.3959 Val ROC AUC: 0.8864
Epoch [15/30] Train Loss: 0.3904 Val Loss: 0.3981 Val ROC AUC: 0.8855
Epoch [20/30] Train Loss: 0.3849 Val Loss: 0.3991 Val ROC AUC: 0.8857
Epoch [25/30] Train Loss: 0.3831 Val Loss: 0.4005 Val ROC AUC: 0.8853
Epoch [30/30] Train Loss: 0.3793 Val Loss: 0.4046 Val ROC AUC: 0.8839

Best ROC AUC on val: 0.8865


In [13]:
scaler_rob = RobustScaler()
train_loader_rob = create_dataloaders(df_train, feature_cols, target_col,
                                      batch_size=batch_size, scaler=scaler_rob,
                                      sampler_use=False, shuffle=True)
val_loader_rob = create_dataloaders(df_val, feature_cols, target_col,
                                    batch_size=batch_size, scaler=scaler_rob,
                                    sampler_use=False, shuffle=False)

model_baseline_rob = BaselineNN(input_dim=len(feature_cols), hidden_dim=64, dropout=0.2)
model_baseline_rob, history_baseline_rob = train_model(
    model_baseline_rob,
    train_loader_rob,
    val_loader_rob,
    epochs=30,
    lr=1e-3,
    use_focal=False
)


Epoch [1/30] Train Loss: 0.4622 Val Loss: 0.4062 Val ROC AUC: 0.8781
Epoch [5/30] Train Loss: 0.4048 Val Loss: 0.3988 Val ROC AUC: 0.8836
Epoch [10/30] Train Loss: 0.3975 Val Loss: 0.3976 Val ROC AUC: 0.8859
Epoch [15/30] Train Loss: 0.3918 Val Loss: 0.3945 Val ROC AUC: 0.8874
Epoch [20/30] Train Loss: 0.3864 Val Loss: 0.3953 Val ROC AUC: 0.8876
Epoch [25/30] Train Loss: 0.3843 Val Loss: 0.3957 Val ROC AUC: 0.8866
Epoch [30/30] Train Loss: 0.3808 Val Loss: 0.3946 Val ROC AUC: 0.8878

Best ROC AUC on val: 0.8883


## 6.2 Углублённая модель с Focal Loss и балансировкой классов

Мы уже ранее сделали несколько экспериментов:
- Проверили разные скейлеры (Standard, Robust, MinMax, MaxAbs) и увидели, что **RobustScaler** даёт лучшие результаты по ROC AUC.
- Попробовали базовую архитектуру сети (BaselineNN).

Теперь усложним модель, добавим:
1. **BatchNorm1d** — для стабилизации обучения,
2. **LeakyReLU** вместо ReLU,
3. **Dropout** более высокий (0.4),
4. **Focal Loss** — улучшает работу на несбалансированных данных,
5. **WeightedRandomSampler** — ещё один инструмент борьбы с дисбалансом.

Также реализуем:
- **Early Stopping** (параметр `patience`), чтобы при отсутствии улучшения по ROC AUC валидация останавливалась досрочно,
- **OneCycleLR** (опционально) — динамическая корректировка `lr` в течение эпох, что может ускорить сходимость и улучшить результат.


In [26]:

class SmokingDataset(Dataset):
    def __init__(self, df, feature_cols, target_col='smoking'):
        self.df = df.reset_index(drop=True)
        self.features = self.df[feature_cols].values
        self.targets = self.df[target_col].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = self.features[idx, :].astype(np.float32)
        y = int(self.targets[idx])
        return torch.tensor(X), torch.tensor(y)


def create_dataloaders(df, feature_cols, target_col,
                       batch_size=32, scaler=None,
                       sampler_use=False, shuffle=False):
    """
    Создаёт DataLoader для указанных данных.
      - Если scaler != None, применяет его к feature_cols.
      - Если sampler_use=True, добавляет WeightedRandomSampler.
    """
    # Если хотим применить скейлер (например, RobustScaler)
    if scaler is not None:
        df_temp = df.copy()
        df_temp[feature_cols] = scaler.fit_transform(df_temp[feature_cols])
    else:
        df_temp = df

    dataset = SmokingDataset(df_temp, feature_cols, target_col)

    if sampler_use:
        # считаем веса классов
        labels = df_temp[target_col].values
        class_weights = 1. / torch.tensor(
            [(labels == t).sum() for t in np.unique(labels)], dtype=torch.float
        )
        sample_weights = class_weights[labels]
        sampler = torch.utils.data.WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )
        dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return dataloader





### 6.2.2 Определяем Focal Loss

`FocalLoss` более агрессивно штрафует ошибки на сложно классифицируемых объектах, что улучшает обучение при дисбалансе классов.


In [27]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        else:
            return focal_loss


In [28]:
class AdvancedNN(nn.Module):
    """
    Углублённая модель:
    - 3 полносвязных блока (512 -> 256 -> 128)
    - BatchNorm + Dropout + LeakyReLU
    - Выход 2 класса
    """
    def __init__(self, input_dim, hidden_size=512, dropout=0.4):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.act1 = nn.LeakyReLU()
        self.drop1 = nn.Dropout(dropout)

        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.bn2 = nn.BatchNorm1d(hidden_size // 2)
        self.act2 = nn.LeakyReLU()
        self.drop2 = nn.Dropout(dropout)

        self.fc3 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.bn3 = nn.BatchNorm1d(hidden_size // 4)
        self.act3 = nn.LeakyReLU()
        self.drop3 = nn.Dropout(dropout)

        self.out = nn.Linear(hidden_size // 4, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.drop2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.act3(x)
        x = self.drop3(x)

        x = self.out(x)
        return x


### 6.2.4 Функция обучения `train_model` с Early Stopping и OneCycleLR

- Если `val_loader=None`, обучение идёт без валидации (например, когда хотим обучиться на всём датасете).
- Если `use_onecycle=True`, используем шедулер `OneCycleLR`.
- Параметр `patience` (Early Stopping): если улучшения ROC AUC нет больше `patience` эпох, мы завершаем обучение досрочно.


In [29]:
def train_model(
    model,
    train_loader,
    val_loader=None,        # Может быть None, если хотим обучать без валидации
    epochs=20,
    lr=1e-3,
    use_focal=False,
    alpha_focal=0.25,
    gamma_focal=2.0,
    patience=5,            # Early Stopping: кол-во эпох без улучшения
    use_onecycle=False
):
    """
    Функция обучения модели, возвращает:
    - обученную модель (лучшая по валидации)
    - (train_history, val_history, best_auc)
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Шедулер OneCycleLR (только при наличии val_loader, иначе смысла мало)
    if use_onecycle and (val_loader is not None):
        steps_per_epoch = len(train_loader)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=lr,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch
        )
    else:
        scheduler = None

    # Критерий
    if use_focal:
        criterion = FocalLoss(alpha=alpha_focal, gamma=gamma_focal)
    else:
        criterion = nn.CrossEntropyLoss()

    model = model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_auc = 0.0

    train_history = []
    val_history = []
    no_improve_epochs = 0

    for epoch in range(epochs):
        # === TRAINING ===
        model.train()
        train_losses = []
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()

            train_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        train_history.append(train_loss)

        # === VALIDATION ===
        if val_loader is not None:
            model.eval()
            val_losses = []
            all_probs = []
            all_labels = []
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch = X_batch.to(device)
                    y_batch = y_batch.to(device)

                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    val_losses.append(loss.item())

                    # вероятности для класса 1
                    probs = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                    all_probs.extend(probs)
                    all_labels.extend(y_batch.cpu().numpy())

            val_loss = np.mean(val_losses)
            val_history.append(val_loss)
            val_auc = roc_auc_score(all_labels, all_probs)

            if (epoch == 0) or ((epoch+1) % 5 == 0) or (epoch == epochs-1):
                print(f"Epoch [{epoch+1}/{epochs}] ",
                      f"Train Loss: {train_loss:.4f} ",
                      f"Val Loss: {val_loss:.4f} ",
                      f"Val ROC AUC: {val_auc:.4f}")

            # Early Stopping
            if val_auc > best_auc:
                best_auc = val_auc
                best_model_wts = copy.deepcopy(model.state_dict())
                no_improve_epochs = 0
            else:
                no_improve_epochs += 1
                if patience is not None and no_improve_epochs >= patience:
                    print(f"Early stopping on epoch {epoch+1}. Best AUC = {best_auc:.4f}")
                    break

        else:
            # Если нет val_loader, мы не можем вычислять AUC и не делаем ранней остановки
            val_auc = 0.0

    if val_loader is not None:
        print(f"\nBest ROC AUC on val: {best_auc:.4f}")
        model.load_state_dict(best_model_wts)
    else:
        print("Training finished (no validation used).")

    return model, (train_history, val_history, best_auc)


### 6.2.5 Обучение с учётом, что `RobustScaler` дал лучший результат

Далее мы:
1. Делим `train_data` на `df_train`/`df_val` (stratify по целевой).
2. Создаём DataLoader с `RobustScaler` и `WeightedRandomSampler` на `df_train`.
3. Создаём DataLoader с `RobustScaler` (без самплера) на `df_val`.
4. Инициализируем модель `AdvancedNN` и обучаем, используя:
   - Focal Loss,
   - OneCycleLR (при желании),
   - Early Stopping.

После этого анализируем лучшую ROC AUC на валидации.


In [31]:
# Разделим данные на train/val
df_train, df_val = train_test_split(
    train_data,
    test_size=0.2,
    random_state=42,
    stratify=train_data[target_col]
)

# Создание DataLoader'ов
train_loader_adv = create_dataloaders(
    df_train,
    feature_cols=num_cols,
    target_col=target_col,
    batch_size=32,
    scaler=RobustScaler(),  # лучший по предыдущим экспериментам
    sampler_use=True,       # включаем балансировку
    shuffle=False
)
val_loader_adv = create_dataloaders(
    df_val,
    feature_cols=num_cols,
    target_col=target_col,
    batch_size=32,
    scaler=RobustScaler(),
    sampler_use=False,
    shuffle=False
)

# Инициализация модели
model_adv = AdvancedNN(input_dim=len(num_cols), hidden_size=512, dropout=0.4)

# Обучение модели
model_adv, history_adv = train_model(
    model=model_adv,
    train_loader=train_loader_adv,
    val_loader=val_loader_adv,
    epochs=50,
    lr=1e-3,
    use_focal=True,
    alpha_focal=0.25,
    gamma_focal=2.0,
    patience=10,        # если за 10 эпох AUC не растёт, остановимся
    use_onecycle=True  # динамическое изменение lr
)

print("Лучшая ROC AUC:", history_adv[2])


Epoch [1/50]  Train Loss: 0.0430  Val Loss: 0.0290  Val ROC AUC: 0.8653
Epoch [5/50]  Train Loss: 0.0299  Val Loss: 0.0269  Val ROC AUC: 0.8830
Epoch [10/50]  Train Loss: 0.0280  Val Loss: 0.0261  Val ROC AUC: 0.8849
Epoch [15/50]  Train Loss: 0.0273  Val Loss: 0.0276  Val ROC AUC: 0.8853
Epoch [20/50]  Train Loss: 0.0271  Val Loss: 0.0266  Val ROC AUC: 0.8856
Epoch [25/50]  Train Loss: 0.0267  Val Loss: 0.0271  Val ROC AUC: 0.8874
Early stopping on epoch 29. Best AUC = 0.8876

Best ROC AUC on val: 0.8876
Лучшая ROC AUC: 0.8875711816814292


### 6.2.6 Обучение на всём train (для финального сабмита)

Чтобы использовать **все** данные для обучения, объединяем `df_train` и `df_val`.  
Но в этом случае валидации не будет
Поэтому `val_loader=None`.


In [36]:
# Объединяем train+val
df_full_train = pd.concat([df_train, df_val], axis=0).reset_index(drop=True)

# DataLoader на полном объёме
full_train_loader = create_dataloaders(
    df_full_train,
    feature_cols=num_cols,
    target_col=target_col,
    batch_size=32,
    scaler=RobustScaler(),
    sampler_use=True,  # балансируем
    shuffle=False
)

# Инициализируем новую такую же модель
model_final = AdvancedNN(input_dim=len(num_cols), hidden_size=512, dropout=0.4)

# Обучаем без валидации
model_final, _ = train_model(
    model=model_final,
    train_loader=full_train_loader,
    val_loader=None,     # нет валидации
    epochs=20,
    lr=1e-3,
    use_focal=True,
    alpha_focal=0.25,
    gamma_focal=2.0,
    patience=None,       # Early Stopping не работает без валидации
    use_onecycle=False
)


Training finished (no validation used).


### 6.2.7 Предсказание на тесте и формирование сабмита

Загружаем и обрабатываем `test_data`, предсказываем вероятности класса 1, сохраняем в файл `submission.csv`.


In [38]:
# Готовим тест к предсказанию
test_features = test_data.drop('id', axis=1).copy()

# Обучаем новый  RobustScaler на *всех* тренировочных данных
scaler_final = RobustScaler()
scaler_final.fit(df_full_train[num_cols])

test_features[num_cols] = scaler_final.transform(test_features[num_cols])

X_test_tensor = torch.tensor(test_features[num_cols].values, dtype=torch.float32)
model_final.eval()
model_final.to(device)

preds = []
with torch.no_grad():
    for i in range(0, len(X_test_tensor), 32):
        batch = X_test_tensor[i:i+32].to(device)
        outputs = model_final(batch)
        # Берём вероятность класса 1
        probs = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        preds.extend(probs)

submission = sample_submission.copy()
submission['smoking'] = preds
submission.to_csv('submission_drop_out04.csv', index=False)

print("Пример первых строк сабмита:")
print(submission.head(10))


Пример первых строк сабмита:
      id   smoking
0  15000  0.405092
1  15001  0.059327
2  15002  0.215243
3  15003  0.501289
4  15004  0.108295
5  15005  0.539450
6  15006  0.528859
7  15007  0.124496
8  15008  0.580619
9  15009  0.091712
